In [ ]:
import numpy as np
import soundfile as sf
import pandas as pd
import easier as ezr
from IPython.display import Audio
import holoviews as hv
from scipy.signal import correlate, correlation_lags
from pandashells.lib.lomb_scargle_lib import lomb_scargle
hv.extension('bokeh')

In [ ]:
%opts Curve [width=800 height=400]

In [ ]:
def spread(y, delays, dt=1, inverse=False):
    """
    y: numpy array of input data
    delays: iterable of delay values
    dt: time beteween samples (1 / sample_rate)
    """
    if not hasattr(delays, '__iter__'):
        raise ValueError('delays argument must be iterable')
    
    # Create an array of frequencies that fft will generate
    f = np.fft.fftfreq(len(y), d=dt)
    
    phase_arrays = [np.ones_like(f)]
    for delay in delays:
        phase_arrays.append(
            np.exp(-2j * np.pi * delay * f)
        )
    
    # Create a phase matrix that has size = (num_delays, len(y))
    phi_matrix = np.stack(phase_arrays, axis=0)
    # phi_matrix = phi_matrix / np.sqrt(len(delays))
    phi_matrix = phi_matrix / len(delays)
    
    # Create the series to convolve with
    phi = np.sum(phi_matrix, axis=0).flatten()
    
    
    # Take the fft of the input
    Y = np.fft.fft(y)
    
    # (de)convolve
    if inverse:
        Y_out = Y / phi
    else:
        Y_out = phi * Y
    
    # Convert back to time domain
    y_out = np.fft.ifft(Y_out).real
    
    # y_out = y_out / np.max(np.abs(y_out))
    
    return y_out

In [ ]:
path = './data/websdr_recording_2022-09-27T00_17_15Z_7160.6kHz.wav'
# path = './1919-142785-0010.flac'                                                  
noise, samplerate_noise = sf.read(path)   
samplerate_noise_interp = 2 * samplerate_noise

noise = noise - np.mean(noise)
noise = noise / np.std(noise)
dt_noise = np.round(1 / samplerate_noise, 7)
t_noise = dt_noise * np.arange(len(noise))
t_noise_interp = .5 * dt_noise  * np.arange(len(noise) * 2)
noise_interp = np.interp(t_noise_interp, t_noise, noise)

In [ ]:
Audio(noise_interp, rate=samplerate_noise_interp)

In [ ]:
path = './data/LibriSpeech/dev-clean/1272/128104/1272-128104-0009.flac'
# path = './1919-142785-0010.flac'                                                  
data, samplerate = sf.read(path)   

data = data - np.mean(data)
# data = data / np.std(data)
data = data / np.max(np.abs(data))
dt = np.round(1 / samplerate, 7)
t = dt * np.arange(len(data))


In [ ]:
Audio(data, rate=samplerate)

In [ ]:
num_lags = 5
base_lag_index = 2*1600  #1600=.1
base_lag_time = dt * base_lag_index
pad_len = num_lags * base_lag_index

delays = base_lag_time * np.arange(1, num_lags + 1)
delays = base_lag_time * np.array([1, 3, 5, 7, 11, 13, 17])

# ###
# data = np.zeros_like(data)
# data[len(data) // 2] = 1
# #####

data_pad = np.concatenate([data, np.zeros(pad_len)])
t_pad = dt * np.arange(len(data_pad))
sig = spread(data_pad, dt=dt, delays=delays)
sig = sig / np.max(np.abs(sig))
recovered = spread(sig, dt=dt, delays=delays, inverse=True)


In [ ]:
c_data = hv.Curve((t_pad, data_pad), label='data').options(color=ezr.cc[0], alpha=.5)
c_sig = hv.Curve((t_pad, sig), label='signal').options(color=ezr.cc[1], alpha=.5)
c_rec = hv.Curve((t_pad, recovered), label='recovered').options(color=ezr.cc[2], alpha=.5)
c_data * c_sig #* c_rec

In [ ]:
Audio(data_pad, rate=samplerate)

In [ ]:
Audio(sig, rate=samplerate)

In [ ]:
noise_amplitude = .1



noisy_data = data + noise_amplitude * noise_interp[:len(data)]
noisy_signal = sig + noise_amplitude * noise_interp[:len(sig)]
recovered_noisy_data = spread(noisy_signal, dt=dt, delays=delays, inverse=True)
t_noise_interp = t_noise_interp[:len(noisy_data)]
recovered_noisy_data = recovered_noisy_data[:len(noisy_data)]


In [ ]:
Audio(noisy_data, rate=samplerate)

In [ ]:
Audio(recovered_noisy_data, rate=samplerate)

In [ ]:
noisy_data.shape, t_noise_interp.shape

In [ ]:
c_data = hv.Curve((t_noise_interp, noisy_data), label='data').options(color=ezr.cc[0], alpha=.5)
c_sig = hv.Curve((t_pad, sig), label='signal').options(color=ezr.cc[2], alpha=.5)
c_rec = hv.Curve((t_noise_interp, recovered_noisy_data), label='recovered').options(color=ezr.cc[1], alpha=.5)
c_data * c_rec
# c_data

In [ ]:
recovered_noisy_data.shape